In [62]:
import string
import numpy as np
import csv
import pandas as pd


In [63]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if substring in big_string:
            return substring
    #print(substring)
    return np.nan

In [64]:
def get_data_from_file(file_location):
    file_content=[]

    with open(file_location) as csvfile:
        file = csv.reader(csvfile)
        for eachline in file:
            file_content.append(eachline)
    
    df=pd.DataFrame(file_content)
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header
    return df

In [65]:
 #Data Engineering

def process_train_data(df):
    
    ##Cherbourg 1
    ##Queenstown 2
    ##Southampton 3

    df['Embarked'] = df['Embarked'].replace(to_replace = "C",value = 1)
    df['Embarked'] = df['Embarked'].replace(to_replace = "Q",value = 2)
    df['Embarked'] = df['Embarked'].replace(to_replace = "S",value = 3)
    
    
    df['Fare'] = pd.to_numeric(df['Fare'])
    df['Age'] = pd.to_numeric(df['Age'])
    
    df['FamilySize'] = pd.to_numeric(df['SibSp'])+pd.to_numeric(df['Parch'])
    
    cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
    df['Deck']=df['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
    
    df = df.replace(to_replace = np.NaN, value=0)
    df = df.replace(to_replace = '',value = 0)
    df = df.replace(to_replace = "male", value = 1)
    df = df.replace(to_replace = "female",value = 2)
    
    new_df = df[['PassengerId','Pclass','Sex','Age','Embarked','Fare','FamilySize','Survived']]
    
    return new_df

In [66]:
 #Data Engineering

def process_test_data(df):
    
    ##Cherbourg 1
    ##Queenstown 2
    ##Southampton 3

    df['Embarked'] = df['Embarked'].replace(to_replace = "C",value = 1)
    df['Embarked'] = df['Embarked'].replace(to_replace = "Q",value = 2)
    df['Embarked'] = df['Embarked'].replace(to_replace = "S",value = 3)
    
    
    df['Fare'] = pd.to_numeric(df['Fare'])
    df['Age'] = pd.to_numeric(df['Age'])
    
    df['FamilySize'] = pd.to_numeric(df['SibSp'])+pd.to_numeric(df['Parch'])
    
    cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
    df['Deck']=df['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
    
    df = df.replace(to_replace = np.NaN, value=0)
    df = df.replace(to_replace = '',value = 0)
    df = df.replace(to_replace = "male", value = 1)
    df = df.replace(to_replace = "female",value = 2)
    
    new_df = df[['PassengerId','Pclass','Sex','Age','Embarked','Fare','FamilySize']]
    
    return new_df

In [67]:
def write_output(prediction,file_name):
    with open(file_name,'w', newline='') as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',')
        csvWriter.writerows(prediction)

In [68]:
new_df = get_data_from_file('./train.csv')
new_df = process_train_data(new_df)

In [69]:
data = new_df[['PassengerId','Pclass','Sex','Age','Embarked','Fare','FamilySize']]

In [70]:
prediction = new_df[['Survived']]

In [71]:
from sklearn import svm

clf = svm.SVC(kernel='linear',C=0.01)

In [72]:
clf.fit(data,prediction)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [73]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(max_depth=7, random_state=2)
clf_rf.fit(data,prediction)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=2, verbose=0, warm_start=False)

In [74]:

def train_data_test_SVC(data):
    data_train = data[0:int(0.8*len(data))]
    prediction_train = prediction[0:int(0.8*len(data))]
    data_test = data[int(0.8*len(data)):]
    prediction_test = prediction[int(0.8*len(data)):]
    
    from sklearn import svm

    clf = svm.SVC(kernel='linear',C=0.01)
    clf.fit(data_train,prediction_train)
    
    pred_sur = clf.predict(data_test)
    
    from sklearn.metrics import accuracy_score

    print(accuracy_score(pred_sur,prediction_test))
    

train_data_test_SVC(data)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.837988826816


In [75]:
def train_data_test_RF(data):
    data_train = data[0:int(0.8*len(data))]
    prediction_train = prediction[0:int(0.8*len(data))]
    data_test = data[int(0.8*len(data)):]
    prediction_test = prediction[int(0.8*len(data)):]
    
    from sklearn.ensemble import RandomForestClassifier

    clf_rf = RandomForestClassifier(max_depth=7, random_state=2)
    clf_rf.fit(data_train,prediction_train)
    
    pred_sur = clf_rf.predict(data_test)
    
    from sklearn.metrics import accuracy_score

    print(accuracy_score(pred_sur,prediction_test))
    

train_data_test_RF(data)

0.860335195531


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


In [76]:
original_test_data = get_data_from_file('./test.csv')
original_test_data = process_test_data(original_test_data)

In [77]:
##Linear SVM
prediction_test_data_svc = clf.predict(original_test_data)

In [78]:
##Linear SVM

id_num=0
prediction=[]
titles=[]
titles.append("PassengerId")
titles.append("Survived")
prediction.append(titles)
ids = original_test_data['PassengerId']

for x in prediction_test_data_svc:
    id_num+=1
    id_prediction=[]
    id_prediction.append(int(ids[id_num]))
    id_prediction.append(int(x))
    prediction.append(id_prediction)
    
write_output(prediction,"output_svm_linear.csv")

In [79]:
prediction_test_data_rf = clf_rf.predict(original_test_data)

In [80]:
##Random Forest

id_num=0
prediction=[]
titles=[]
titles.append("PassengerId")
titles.append("Survived")
prediction.append(titles)
ids = original_test_data['PassengerId']

for x in prediction_test_data_rf:
    id_num+=1
    id_prediction=[]
    id_prediction.append(int(ids[id_num]))
    id_prediction.append(int(x))
    prediction.append(id_prediction)
    
write_output(prediction,"output_random_forest.csv")